In [2]:
import sys
import importlib

sys.path.append("../")

from src import utils
from src.utils import *

from src import plot_utils
from src import graph_utils
from src import inpaint_utils
from src import fiberatlas_utils

## Goal

The goal of this notebook is to do inpainting. Allowing for negative weightings of bundle to generate bundles, and allowing as well negatively connected bundles to exist

## Description

In [3]:
scale = 3
connFilename = f'../../atlas_data/fiber_atlas/probconnatlas/wm.connatlas.scale{scale}.h5'
hf = h5py.File(connFilename, 'r')

centers = np.array(hf.get('header').get('gmcoords'))
nsubject = hf.get('header').get('nsubjects')[()]
dim = hf.get('header').get('dim')[()]
fiber_affine = hf.get('header').get('affine')[()]

gmregions_names = hf.get('header').get('gmregions')[()]
nb_regions = gmregions_names.shape[0]

gm_mask_subj = nib.load('../../atlas_data/moviedata_fMRI_eg/gm_mask_subj7.nii').get_fdata() 
wm_mask_subj = (gm_mask_subj + 1) % 2


consistency_view = fiberatlas_utils.get_aggprop(hf, 'consistency')
length_view = fiberatlas_utils.get_aggprop(hf, 'length')
nbStlines_view = fiberatlas_utils.get_aggprop(hf, 'numbStlines')
nb_regions = consistency_view.shape[0]

## Regressing Bundle Activities

In [4]:
# NOTE: consider bundles that appear at least in 30 % of the subjects
thresh_subjapp = int(np.ceil(nsubject * 0.1)) 

In [5]:
# GENERATING ORIGINAL - without caring about voxel percentage
# Generating the X samples and the y samples
# 1. Careful as well to remove the auto-correlation in the diagonal
# 2. Raster scan parsing meaning that it is the activity of (R0,R1) -> (R0,R2) -> (R0,R3) etc...
# Define matrix of end points on cortex
X = []
bundles_labels = []
for i in tqdm(range(1,nb_regions + 1)):
    for j in range(i,nb_regions + 1):
        tmp = fiberatlas_utils.get_bundles_betweenreg(hf, i, j, verbose=False)
        if tmp is None: continue
        if np.sum(tmp[:,3] >= (thresh_subjapp)) == 0: continue
        bundles_labels.append((i,j))
        vec = np.zeros(nb_regions)
        vec[i-1] = 1.0
        vec[j-1] = 1.0
        X.append(vec)

X = np.array(X)

100%|██████████| 243/243 [46:55<00:00, 11.59s/it]   


In [6]:
root = '../../atlas_data/fiber_atlas/yasser_datacomp/volspams_compress/'

atlas_of_interest = f'compresslausanne2018.scale{scale}.sym.corrected.ctx+subc.volspams.nii.gz'

prob_regions, prob_affine = (nib.load(root + atlas_of_interest).get_fdata()[:,:,:,1:], 
                             nib.load(root + atlas_of_interest).affine)

Xp = []
bundles_labels = []
for i in tqdm(range(1,nb_regions + 1)):
    for j in range(i,nb_regions + 1):
        tmp = fiberatlas_utils.get_bundles_betweenreg(hf, i, j, verbose=False)
        if tmp is None: continue
        if np.sum(tmp[:,3] >= (thresh_subjapp)) == 0: continue
        bundle_coords = tmp[:,[0,1,2]]

        prob_vox = np.zeros_like(prob_regions[:,:,:,0])
        prob_vox[bundle_coords[:,0], bundle_coords[:,1], bundle_coords[:,2]] = 1.0

        region_i = prob_regions[:,:,:,i-1]
        region_j = prob_regions[:,:,:,j-1]

        bundle_proba_i = (region_i * prob_vox)
        bproba_i = bundle_proba_i[bundle_proba_i!=0].mean()
        bundle_proba_j = (region_j * prob_vox)
        bproba_j = bundle_proba_j[bundle_proba_j!=0].mean()

        bundles_labels.append((i,j))
        vec = np.zeros(nb_regions)
        vec[i-1] = bproba_i
        vec[j-1] = bproba_j
        Xp.append(vec)

Xp = np.array(Xp)
Xp = np.nan_to_num(Xp)

 44%|████▍     | 108/243 [15:45<11:51,  5.27s/it]/var/folders/9t/60tqjhtx2h55525lstv8nsc40000gn/T/ipykernel_59200/2900515649.py:24: RuntimeWarning: Mean of empty slice.
  bproba_i = bundle_proba_i[bundle_proba_i!=0].mean()
/Users/mikichan/miniconda3/envs/microstruct_atlasing/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 243/243 [1:03:44<00:00, 15.74s/it]  


In [7]:
region_ftimecourse = load(f"../../atlas_data/moviedata_fMRI_eg/yasseratlased_fmri/ftimecourse_95_scale{scale}.pkl")
regions_in_voxels = load(f'../../atlas_data/fiber_atlas/regions95_voxels_scale{scale}.pkl')[:,:,:,1:]

In [8]:
# Regressing with L1-L2
regularizers = np.sort(np.concatenate([np.logspace(0,5,6), np.logspace(0,5,6) / 2]))

perf_ridge = {'coefs': [],'scores': [], 'intercepts': []}
for k in tqdm(range(len(regularizers))):
    coefs, scores, intercepts = inpaint_utils.regress_linear(X, region_ftimecourse, 
                                                            regularizers[k], ridgeflag=True, verbose=False)
    perf_ridge['coefs'].append(coefs)
    perf_ridge['scores'].append(scores)
    perf_ridge['intercepts'].append(intercepts)

perf_lasso = {'coefs': [],'scores': [], 'intercepts': []}
for k in tqdm(range(len(regularizers))):
    coefs, scores, intercepts = inpaint_utils.regress_linear(X, region_ftimecourse, 
                                                            regularizers[k], ridgeflag=False, verbose=False)
    perf_lasso['coefs'].append(coefs)
    perf_lasso['scores'].append(scores)
    perf_lasso['intercepts'].append(intercepts)

100%|██████████| 12/12 [10:15<00:00, 51.25s/it]


In [9]:
save(f"../resources/weights_regressors_activity/weighted_bundle_activity_lasso{thresh_subjapp}_scale{scale}.pkl", perf_lasso)
save(f"../resources/weights_regressors_activity/weighted_bundle_activity_ridge{thresh_subjapp}_scale{scale}.pkl", perf_ridge)

In [10]:
# spatial graph defining
bundle_graph = np.zeros((X.shape[0], X.shape[0]))
for k in range(X.shape[0]):
    avect1 = X[k]
    for s in range(X.shape[0]):
        if s == k: continue
        avect2 = X[s]
        if np.abs(avect1 - avect2).sum() <= 2:
            bundle_graph[k,s] = 1.0
            bundle_graph[s,k] = 1.0

# temporal graph defining
cycle = graph_utils.make_cycle(region_ftimecourse.shape[-1])

Ls = graph_utils.compute_directed_laplacian(bundle_graph)
Lt = graph_utils.compute_directed_laplacian(cycle)

In [11]:
Xmult = np.array([Xp.T for _ in range(region_ftimecourse.shape[-1])])

In [12]:
ps = [(0,0), (1e-2,1e-2), (1e-1,1e-1), (1,1), (10,10), (100,100)]
for (p1,p2) in ps:
    bundle_opt, logs = inpaint_utils.optimize_lreg(Xmult, region_ftimecourse, Ls=Ls, Lt=Lt, 
                                               verbose=True, num_epochs=200, logging=True, p1=p1, p2=p2, lr=1)

    save(f"../resources/weights_regressors_activity/weighted_bundle_activity_timevertex{thresh_subjapp}_scale{scale}_p1-{p1}_p2-{p2}.pkl", bundle_opt)

100%|██████████| 200/200 [9:27:55<00:00, 170.38s/it]    


Losses are decomposed into:
generic loss=tensor([0.7468])
spatialloss=tensor([10657166.])
temporalloss=tensor([12104.2012])
sumloss=tensor([0.7468])


100%|██████████| 200/200 [1:14:52<00:00, 22.46s/it]


Losses are decomposed into:
generic loss=tensor([12787.6006])
spatialloss=tensor([3107269.])
temporalloss=tensor([5381.7168])
sumloss=tensor([43914.1055])


100%|██████████| 200/200 [1:13:51<00:00, 22.16s/it]


...

Losses are decomposed into:
generic loss=tensor([86394.8359])
spatialloss=tensor([388395.1562])
temporalloss=tensor([1479.9309])
sumloss=tensor([476269.9375])


100%|██████████| 200/200 [1:11:37<00:00, 21.49s/it]


Losses are decomposed into:
generic loss=tensor([92826.6953])
spatialloss=tensor([388430.1562])
temporalloss=tensor([669.5716])
sumloss=tensor([3983824.])


100%|██████████| 200/200 [1:41:27<00:00, 30.44s/it]   

Losses are decomposed into:
generic loss=tensor([108807.9062])
spatialloss=tensor([388880.9688])
temporalloss=tensor([483.9220])
sumloss=tensor([39045296.])


In [13]:
# NOTE: Close the opened h5 file
hf.close()